# BERT를 활용한 단일 문장 분류 실습

* datasets
  * huggingface에서 출시한 dataset 라이브러리
  * 원하는 dataset이름 호출을 통해 dataset을 가져옴

In [ ]:
# !pip install transformers
# !pip install datasets

In [ ]:
import torch
import datasets
import sys

* 디바이스 설정

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

* 저장된 dataset list확인

In [ ]:
# 사용가능한 dataset list 불러오기
dataset_list = datasets.list_datasets()

# dataset list 확인
for datas in dataset_list:
    if 'ko' in datas:
        print(datas)

'''
kor_3i4k
kor_hate
kor_ner
kor_nli
kor_nlu
kor_qpair
kor_sae
kor_sarcasm
squad_kor_v1
squad_kor_v2
KETI-AIR/kor_corpora
KETI-AIR/korquad
abwicke/koplo
huggingartists/aikko
huggingartists/boris-grebenshikov
huggingartists/krept-and-konan-bugzy-malone-sl-morisson-abra-cadabra-rv-and-snap-capone
huggingartists/lyapis-trubetskoy
huggingartists/max-korzh
roskoN/dailydialog
roskoN/dstc8-reddit-corpus
toriving/kosimcse
'''

* 'nsmc'
  * 네이버에서 출시한 영화 댓글의 감정 분류 데이터셋

* 단일 문장 분류 task에는 'nsmc' 외에도 'hate', 'sarcasm'을 사용할 수 있음

* dataset은 dictionary 형태로 train과 test가 나뉘어져 있음
  * 'document' : sentence
  * 'label' : 감정(긍정/부정)

In [ ]:
# nsmc 데이터 로드
dataset = datasets.load_dataset('nsmc') # nsmc, hate, sarcasm

# 데이터셋 구조 확인
print(dataset)
'''
DatasetDict({
    train: Dataset({
        features: ['id', 'document', 'label'],
        num_rows: 150000
    })
    test: Dataset({
        features: ['id', 'document', 'label'],
        num_rows: 50000
    })
})
'''

In [ ]:
import pandas as pd

* dataset을 다루기 편한 형태로 변환

In [ ]:
# 필요한 데이터인 document와 label 정보만 pandas라이브러리 DataFrame 형식으로 변환
train_data = pd.DataFrame({"document":dataset['train']['document'], "label":dataset['train']['label'],})
test_data = pd.DataFrame({"document":dataset['test']['document'], "label":dataset['test']['label'],})

In [ ]:
# 데이터셋 갯수 확인
print('학습 데이터셋 : {}'.format(len(train_data)))
# 학습 데이터셋 : 150000
print('테스트 데이터셋 : {}'.format(len(test_data)))
# 테스트 데이터셋 : 50000

In [ ]:
# 데이터셋 내용 확인
train_data[:5]
'''
	                                                                              document	label
0	                                                      아 더빙.. 진짜 짜증나네요 목소리	    0
1	                              흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나	    1
2	                                                    너무재밓었다그래서보는것을추천한다	    0
3	                                     교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정	    0
4	사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...	    1
'''

In [ ]:
test_data[:5]
'''
	                                                                      document	label
0	                                                                         굳 ㅋ	    1
1	                                                          GDNTOPCLASSINTHECLUB	    0
2	                뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아	    0
3	                          지루하지는 않은데 완전 막장임... 돈주고 보기에는....	    0
4	3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??	    0
'''

* 데이터 전처리 과정
  * 중복 데이터 제거

In [ ]:
# 데이터 중복을 제외한 갯수 확인
print("학습데이터 : ",train_data['document'].nunique()," 라벨 : ",train_data['label'].nunique())
# 학습데이터 :  146183  라벨 :  2
print("데스트 데이터 : ",test_data['document'].nunique()," 라벨 : ",test_data['label'].nunique())
# 데스트 데이터 :  49158  라벨 :  2

# 중복 데이터 제거
train_data.drop_duplicates(subset=['document'], inplace= True)
test_data.drop_duplicates(subset=['document'], inplace= True)

# 데이터셋 갯수 확인
print('중복 제거 후 학습 데이터셋 : {}'.format(len(train_data)))
# 중복 제거 후 학습 데이터셋 : 146183
print('중복 제거 후 테스트 데이터셋 : {}'.format(len(test_data)))
# 중복 제거 후 테스트 데이터셋 : 49158

In [ ]:
import numpy as np

* 데이터 전처리
  * null 데이터 제거

In [ ]:
# null 데이터 제거
train_data['document'].replace('', np.nan, inplace=True)
test_data['document'].replace('', np.nan, inplace=True)
train_data = train_data.dropna(how = 'any')
test_data = test_data.dropna(how = 'any')

print('null 제거 후 학습 데이터셋 : {}'.format(len(train_data)))
# null 제거 후 학습 데이터셋 : 146182
print('null 제거 후 테스트 데이터셋 : {}'.format(len(test_data)))
# null 제거 후 테스트 데이터셋 : 49157

In [ ]:
print(train_data['document'][0])
# 아 더빙.. 진짜 짜증나네요 목소리
print(train_data['label'][0])
# 0


* 데이터 전처리
  * outlier 제거

In [ ]:
from matplotlib import pyplot as plt

#학습 리뷰 길이조사
print('학습 문장 최대 길이 :',max(len(l) for l in train_data['document']))
# 학습 문장 최대 길이 : 146
print('학습 문장의 평균 길이 :',sum(map(len, train_data['document']))/len(train_data['document']))
# 학습 문장의 평균 길이 : 35.981338331668745

plt.hist([len(s) for s in train_data['document']], bins=50)
plt.xlabel('length of data')
plt.ylabel('number of data')
plt.show()

* pretrained된 multilingual model 사용

In [ ]:
# Store the model we want to use
from transformers import AutoModel, AutoTokenizer, BertTokenizer
MODEL_NAME = "bert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

* tokenizer에 list 형태로 입력하면 list 형태로 반환됨

In [ ]:
tokenized_train_sentences = tokenizer(
    list(train_data['document']),
    return_tensors="pt",
    padding=True,
    truncation=True,
    add_special_tokens=True,
    )

In [ ]:
print(tokenized_train_sentences[0])
# Encoding(num_tokens=142, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])
print(tokenized_train_sentences[0].tokens)
# ['[CLS]', '아', '더', '##빙', '.', '.', '진', '##짜', '짜', '##증', '##나', '##네', '##요', '목', '##소', '##리', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']
print(tokenized_train_sentences[0].ids)
# [101, 9519, 9074, 119005, 119, 119, 9708, 119235, 9715, 119230, 16439, 77884, 48549, 9284, 22333, 12692, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
print(tokenized_train_sentences[0].attention_mask)
# [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [ ]:
tokenized_test_sentences = tokenizer(
    list(test_data['document']),
    return_tensors="pt",
    padding=True,
    truncation=True,
    add_special_tokens=True,
    )

In [ ]:
train_label = train_data['label'].values
test_label = test_data['label'].values

In [ ]:
print(train_label[0]) # 0

* 실제 model에 입력하기 위한 구조적인 형태로 변환
  * huggingface의 dataset과 model을 사용하는 경우 형태가 대부분 일치함

* `__getitem__()`
  * step 맞는 dataset을 model가져오는 역할

In [ ]:
class SingleSentDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx): 
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)


In [ ]:
train_dataset = SingleSentDataset(tokenized_train_sentences, train_label)
test_dataset = SingleSentDataset(tokenized_test_sentences, test_label)

* `BertForSequenceClassification`
  * 분류를 위해 BERT 위에 부착하는 classicfication을 위한 head를 제공

In [ ]:
from transformers import BertForSequenceClassification, Trainer, TrainingArguments
# 문장 분류를 위해선 BERT 위에 classification을 위한 head를 부착해야 합니다.
# 해당 부분을 transformers에서는 라이브러리 하나만 호출하면 됩니다! :-)

* `warmup_steps`
  * learning rate의 범위
  * learning rate를 조절함

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=1,              # total number of training epochs
    per_device_train_batch_size=32,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=500,
    save_steps=500,
    save_total_limit=2
)

In [ ]:
model = BertForSequenceClassification.from_pretrained(MODEL_NAME)
model.to(device)

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
)

In [ ]:
trainer.train() # 1 epoch에 대략 30분 정도 소요됩니다 :-)

* 성능 평가를 위한 함수

In [ ]:
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

* evaluate가 실행될 때, `compute_metrics`에 해당하는 함수를 불러와서 평가함

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.evaluate(eval_dataset=test_dataset)
'''
{'eval_accuracy': 0.8609150273613118,
 'eval_f1': 0.8648627280453818,
 'eval_loss': 0.3183152973651886,
 'eval_mem_cpu_alloc_delta': 943706,
 'eval_mem_cpu_peaked_delta': 2675650,
 'eval_mem_gpu_alloc_delta': 0,
 'eval_mem_gpu_peaked_delta': 280995328,
 'eval_precision': 0.8452978904257785,
 'eval_recall': 0.8853547003358828,
 'eval_runtime': 194.1152,
 'eval_samples_per_second': 253.236,
 'init_mem_cpu_alloc_delta': 48822,
 'init_mem_cpu_peaked_delta': 18306,
 'init_mem_gpu_alloc_delta': 0,
 'init_mem_gpu_peaked_delta': 0}
'''

* trainer를 사용하지 않고 torch로 학습

In [ ]:
# native training using torch

# model = BertForSequenceClassification.from_pretrained(MODEL_NAME)
# model.to(device)
# model.train()

# train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# optim = AdamW(model.parameters(), lr=5e-5)

# for epoch in range(3):
#     for batch in train_loader:
#         optim.zero_grad()
#         input_ids = batch['input_ids'].to(device)
#         attention_mask = batch['attention_mask'].to(device)
#         labels = batch['labels'].to(device)
#         outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
#         loss = outputs[0]
#         loss.backward()
#         optim.step()

* prediction 함수

In [ ]:
# predict함수
def sentences_predict(sent):
    model.eval()
    tokenized_sent = tokenizer(
            sent,
            return_tensors="pt",
            truncation=True,
            add_special_tokens=True,
            max_length=128
    )
    tokenized_sent.to(device)
    
    with torch.no_grad():# 그라디엔트 계산 비활성화
        outputs = model(
            input_ids=tokenized_sent['input_ids'],
            attention_mask=tokenized_sent['attention_mask'],
            token_type_ids=tokenized_sent['token_type_ids']
            )

    logits = outputs[0]
    logits = logits.detach().cpu().numpy()
    result = np.argmax(logits)
    return result

In [ ]:
print(sentences_predict("영화 개재밌어 ㅋㅋㅋㅋㅋ")) # 1
print(sentences_predict("진짜 재미없네요 ㅋㅋ")) # 0
print(sentences_predict("너 때문에 진짜 짜증나")) # 0
print(sentences_predict("정말 재밌고 좋았어요.")) # 1

* `pipeline`을 사용하면 prediction을 구현하지 않아도 됨

* model을 pipeline에 이미 지정했기 때문에, 테스트할 때 따로 넣어주지 않아도 됨

In [ ]:
from transformers import pipeline

nlp_sentence_classif = pipeline('sentiment-analysis',model=model, tokenizer=tokenizer, device=0)

print(nlp_sentence_classif('영화 개재밌어 ㅋㅋㅋㅋㅋ'))
# [{'label': 'LABEL_1', 'score': 0.7245705723762512}]
print(nlp_sentence_classif('진짜 재미없네요 ㅋㅋ',model= model)) # model 생략 가능
# [{'label': 'LABEL_0', 'score': 0.9946942925453186}]
print(nlp_sentence_classif('너 때문에 진짜 짜증나',model= model)) # model 생략 가능
# [{'label': 'LABEL_0', 'score': 0.9947050213813782}]
print(nlp_sentence_classif('정말 재밌고 좋았어요.',model= model)) # model 생략 가능
# [{'label': 'LABEL_1', 'score': 0.989768385887146}]